In [1]:
import sklearn

import pandas as pd
import numpy as np

In [2]:
print(sklearn.__version__)
print(np.__version__)
print(pd.__version__)

0.23.2
1.18.5
0.23.0


In [3]:
student_df = pd.read_csv('..\\datasets\\2019_PublicDataFile_Students.csv')
student_df.head(5)

,DBN,School Name,Total Parent Response Rate %,Total Teacher Response Rate %,Total Student Response Rate %,Collaborative Teachers Score,Effective School Leadership Score,Rigorous Instruction Score,Supportive Environment Score,Strong Family-Community Ties Score,Trust Score
0,01M015,P.S. 015 ROBERTO CLEMENTE,91,100,NaN,4.10,4.19,4.02,NaN,4.18,3.96
1,01M019,P.S. 019 ASHER LEVY,100,93,NaN,4.53,4.51,4.80,NaN,4.66,3.76
2,01M020,P.S. 020 ANNA SILVER,58,90,NaN,2.71,2.98,1.92,NaN,3.84,3.14
3,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,29,100,95.0,2.69,2.59,2.14,NaN,3.67,2.38
4,01M063,THE STAR ACADEMY - P.S.63,80,100,NaN,NaN,NaN,NaN,NaN,NaN,3.77


In [4]:
student_df.shape

(1829, 11)

In [5]:
student_df.isna().sum()

DBN                                      0
School Name                              0
Total Parent \nResponse Rate %           0
Total Teacher Response Rate %            0
Total Student Response Rate %          720
Collaborative Teachers Score           204
Effective School Leadership Score      204
Rigorous Instruction Score             204
Supportive Environment Score          1829
Strong Family-Community Ties Score     219
Trust Score                             72
dtype: int64

In [6]:
student_df = student_df.dropna(subset=['Trust Score'])

In [7]:
student_df = student_df.drop('Supportive Environment Score', 1)
student_df.isna().sum()

DBN                                     0
School Name                             0
Total Parent \nResponse Rate %          0
Total Teacher Response Rate %           0
Total Student Response Rate %         699
Collaborative Teachers Score          149
Effective School Leadership Score     149
Rigorous Instruction Score            149
Strong Family-Community Ties Score    147
Trust Score                             0
dtype: int64

In [8]:
column_name = 'Strong Family-Community Ties Score'
rigorous_score_mean = student_df[column_name].mean()
student_df[column_name] = student_df[column_name].fillna(rigorous_score_mean)

In [9]:
from sklearn.impute import SimpleImputer
column_name = 'Rigorous Instruction Score'
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp.fit(student_df[column_name].values.reshape(-1, 1))

student_df[column_name] = imp.transform(student_df[column_name].values.reshape(-1, 1))

In [10]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

column_name = 'Collaborative Teachers Score'
imp.fit(student_df[column_name].values.reshape(-1, 1))
student_df[column_name] = imp.transform(student_df[column_name].values.reshape(-1, 1))

column_name = 'Effective School Leadership Score'
imp.fit(student_df[column_name].values.reshape(-1, 1))
student_df[column_name] = imp.transform(student_df[column_name].values.reshape(-1, 1))

student_df.isna().sum()

DBN                                     0
School Name                             0
Total Parent \nResponse Rate %          0
Total Teacher Response Rate %           0
Total Student Response Rate %         699
Collaborative Teachers Score            0
Effective School Leadership Score       0
Rigorous Instruction Score              0
Strong Family-Community Ties Score      0
Trust Score                             0
dtype: int64

In [11]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer # Note: KNNImputer can be used instead of IterativeImputer
imp = IterativeImputer(max_iter=10000, random_state=0)

student_features = student_df.drop(['DBN', 'School Name'], axis=1)
student_nonnumeric = student_df[['DBN', 'School Name']]

student_features.head()

,Total Parent Response Rate %,Total Teacher Response Rate %,Total Student Response Rate %,Collaborative Teachers Score,Effective School Leadership Score,Rigorous Instruction Score,Strong Family-Community Ties Score,Trust Score
0,91,100,NaN,4.100000,4.190000,4.02,4.180000,3.96
1,100,93,NaN,4.530000,4.510000,4.80,4.660000,3.76
2,58,90,NaN,2.710000,2.980000,1.92,3.840000,3.14
3,29,100,95.0,2.690000,2.590000,2.14,3.670000,2.38
4,80,100,NaN,3.672382,3.614857,3.69,3.736845,3.77


In [12]:
imp.fit(student_features)
student_features_arr = imp.transform(student_features)
student_features = pd.DataFrame(student_features_arr, columns=student_features.columns)

student_features.isna().sum()

Total Parent \nResponse Rate %        0
Total Teacher Response Rate %         0
Total Student Response Rate %         0
Collaborative Teachers Score          0
Effective School Leadership Score     0
Rigorous Instruction Score            0
Strong Family-Community Ties Score    0
Trust Score                           0
dtype: int64

In [13]:
student_df = pd.concat([student_features, student_nonnumeric], axis=1)
student_df = student_df.dropna()
student_df.head()

,Total Parent Response Rate %,Total Teacher Response Rate %,Total Student Response Rate %,Collaborative Teachers Score,Effective School Leadership Score,Rigorous Instruction Score,Strong Family-Community Ties Score,Trust Score,DBN,School Name
0,91.0,100.0,98.688697,4.100000,4.190000,4.02,4.180000,3.96,01M015,P.S. 015 ROBERTO CLEMENTE
1,100.0,93.0,102.923773,4.530000,4.510000,4.80,4.660000,3.76,01M019,P.S. 019 ASHER LEVY
2,58.0,90.0,85.158718,2.710000,2.980000,1.92,3.840000,3.14,01M020,P.S. 020 ANNA SILVER
3,29.0,100.0,95.000000,2.690000,2.590000,2.14,3.670000,2.38,01M034,P.S. 034 FRANKLIN D. ROOSEVELT
4,80.0,100.0,95.279381,3.672382,3.614857,3.69,3.736845,3.77,01M063,THE STAR ACADEMY - P.S.63


In [14]:
student_df.isna().sum()

Total Parent \nResponse Rate %        0
Total Teacher Response Rate %         0
Total Student Response Rate %         0
Collaborative Teachers Score          0
Effective School Leadership Score     0
Rigorous Instruction Score            0
Strong Family-Community Ties Score    0
Trust Score                           0
DBN                                   0
School Name                           0
dtype: int64

In [15]:
student_df.shape

(1694, 10)

In [16]:
student_df.to_csv('2019_PublicDataFile_Students_processed01.csv', index=False)